In [1]:
import numpy as np
import pandas as pd

train=pd.read_csv("/Users/saksham/Desktop/train.csv")
x_test=pd.read_csv("/Users/saksham/Desktop/test.csv")
Y_train=np.array(train["label"])
X_train=np.array(train.drop(["label"],axis=1))

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
a=unpickle("/Users/saksham/Desktop/cifar-10-batches-py/test_batch")
y=np.array(a)
y_test=np.array(a[b'labels']).astype(int)

In [3]:
sigma =0.01
W = sigma * np.random.randn(3072,10)

In [4]:
class Svm ():
    def calLoss (self, x, y, reg):

        loss = 0.0
        dW = np.zeros_like(W)

        s = x.dot(W)
        s_yi = s[np.arange(x.shape[0]),y]
        delta = s- s_yi[:,np.newaxis]+1
        loss_i = np.maximum(0,delta)
        loss_i[np.arange(x.shape[0]),y]=0
        loss = np.sum(loss_i)/x.shape[0]
        loss += reg*np.sum(W*W)
        ds = np.zeros_like(delta)
        ds[delta > 0] = 1
        ds[np.arange(x.shape[0]),y] = 0
        ds[np.arange(x.shape[0]),y] = -np.sum(ds, axis=1)

        dW = (1/x.shape[0]) * (x.T).dot(ds)
        dW = dW + (2* reg* W)
        return loss, dW

    def train (self, x, y,W, lr=1e-3, reg=1e-5, iter=100, batchSize=1000, verbose=False):
        lossHistory = []
        for i in range(iter):
            xBatch = None
            yBatch = None

            num_train = np.random.choice(x.shape[0], batchSize)
            xBatch = x[num_train]
            yBatch = y[num_train]
            loss, dW = self.calLoss(xBatch,yBatch,reg)
            W= W - lr * dW
            lossHistory.append(loss)

            if verbose and i % 100 == 0 and len(lossHistory) != 0:
                print ('Loop {0} loss {1}'.format(i, lossHistory[i]))

        return lossHistory

    def predict (self, x,):

        yPred = np.zeros(x.shape[0])

        s = x.dot(W)
        yPred = np.argmax(s, axis=1)
        return yPred
    def calAccuracy (self, x, y):
        acc = 0

        yPred = self.predict(x)
        acc = np.mean(y == yPred)*100

        return acc





In [5]:
classifier=Svm()
print ('Start training Svm classifier')

classifier.train(X_train, Y_train,W, iter=500,verbose=True)
print ('Training acc:   {0}%'.format(classifier.calAccuracy(X_train, Y_train)))

Start training Svm classifier
Loop 0 loss 378.79550076386806
Loop 100 loss 363.65322033797156
Loop 200 loss 367.9325915737177
Loop 300 loss 387.9440037998378
Loop 400 loss 373.7831908928379
Training acc:   9.736%


In [8]:
x_test = np.reshape(x_test, (x_test.shape[0], -1))
x_test = np.hstack([x_test, np.ones((x_test.shape[0], 1))])
x_test=x_test[:,0:3072]


In [9]:
print ('Testing acc:    {0}%'.format(classifier.calAccuracy(x_test, y_test)))

Testing acc:    10.22%
